# **📊 Building a Bookstore DB**

- **Author:** Basel Amr Barakat  
- **Email:** baselamr52@gmail.com  
- **Date:** 2024-1-2  
- **Task Name:** Analysis chinook database
- **Task Number:** 26  
- **Part:**  Introduciton to Data Base Managnment
- **Module:** From Data to AI
- **Submit Number:** 1

### **Description:**
This task involves practicing complex SQL queries to combine, aggregate, and rank data from a database. The goal is to practice INNER JOIN, LEFT JOIN, CTEs (Common Table Expressions), window functions, and query optimization. Students will work with a dataset to calculate customer spending, rank products by sales, and optimize queries for performance.



### **Requirements:**

1. **Requirement 1: Complex Joins and CTEs.**  
   - **Description:**  
     1. use the database (`chinook-database`)
     2. Use `INNER JOIN` and `LEFT JOIN` to combine the `Customer`, `Invoice`, and `InvoiceLine` tables.
     3. Use a `CTE` (Common Table Expression) to calculate the total amount spent by each customer.
     4. Calculate the top 10 customers by total spending.

2. **Requirement 2: Window Functions for Ranking**  
   - **Description:**  
     1.  Use window functions to rank products by total sales (using the InvoiceLine and Track tables).
     2. Calculate the rank of each product by total sales amount.
     3. Identify the top-selling products in the database. Use RANK() or ROW_NUMBER() to rank the top-selling products based on the total sales (quantity sold).

3. **Requirement 3: Indexing and Performance Optimization**  
   - **Description:**  
    1. Identify the most commonly queried columns CustomerId in the Invoice table.
    2. Create indexes on these columns and compare query performance with and without indexing.
    3. Write a query that lists total sales for each customer, and optimize it using indexing.


## **0 Setup and DataBase Connection**

Connect to the DataBase using the following command


```
Connection = sqlite3.connect(database_path)
cursor = connection.cursor()
```



In [40]:
import sqlite3
import pandas as pd
import os
import plotly.express as px
import plotly.io as pio
import time
import plotly.express as px
import plotly.io as pio

In [41]:
# Connect to Chinook Database
database_path = 'Chinook_Sqlite.sqlite'
try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    print("✅ Database connection successful!")
except Exception as e:
    print(f"❌ Failed to connect to the database: {e}")

✅ Database connection successful!


### 0.1 Taverse Through Our DataBase

In [42]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table_name in tables:
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    cols = cursor.fetchall()
    print(f"Columns in {table_name[0]}:")
    for col in cols:
        print(col[1])
    print(f"------------------------------------")

Tables in the database:
Columns in Album:
AlbumId
Title
ArtistId
------------------------------------
Columns in Artist:
ArtistId
Name
------------------------------------
Columns in Customer:
CustomerId
FirstName
LastName
Company
Address
City
State
Country
PostalCode
Phone
Fax
Email
SupportRepId
------------------------------------
Columns in Employee:
EmployeeId
LastName
FirstName
Title
ReportsTo
BirthDate
HireDate
Address
City
State
Country
PostalCode
Phone
Fax
Email
------------------------------------
Columns in Genre:
GenreId
Name
------------------------------------
Columns in Invoice:
InvoiceId
CustomerId
InvoiceDate
BillingAddress
BillingCity
BillingState
BillingCountry
BillingPostalCode
Total
------------------------------------
Columns in InvoiceLine:
InvoiceLineId
InvoiceId
TrackId
UnitPrice
Quantity
------------------------------------
Columns in MediaType:
MediaTypeId
Name
------------------------------------
Columns in Playlist:
PlaylistId
Name
--------------------------

In [43]:
# Display All Table Structures
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection)

for table_name in tables['name']:
    print(f"🔍 Structure of Table: {table_name}")
    table_info = pd.read_sql_query(f"PRAGMA table_info('{table_name}');", connection)
    display(table_info)
    print("----------------------------------------------------------------")


🔍 Structure of Table: Album


,cid,name,type,notnull,dflt_value,pk
0,0,AlbumId,INTEGER,1,None,1
1,1,Title,NVARCHAR(160),1,None,0
2,2,ArtistId,INTEGER,1,None,0


----------------------------------------------------------------
🔍 Structure of Table: Artist


,cid,name,type,notnull,dflt_value,pk
0,0,ArtistId,INTEGER,1,None,1
1,1,Name,NVARCHAR(120),0,None,0


----------------------------------------------------------------
🔍 Structure of Table: Customer


...

----------------------------------------------------------------
🔍 Structure of Table: PlaylistTrack


,cid,name,type,notnull,dflt_value,pk
0,0,PlaylistId,INTEGER,1,None,1
1,1,TrackId,INTEGER,1,None,2


----------------------------------------------------------------
🔍 Structure of Table: Track


,cid,name,type,notnull,dflt_value,pk
0,0,TrackId,INTEGER,1,None,1
1,1,Name,NVARCHAR(200),1,None,0
2,2,AlbumId,INTEGER,0,None,0
3,3,MediaTypeId,INTEGER,1,None,0
4,4,GenreId,INTEGER,0,None,0
5,5,Composer,NVARCHAR(220),0,None,0
6,6,Milliseconds,INTEGER,1,None,0
7,7,Bytes,INTEGER,0,None,0
8,8,UnitPrice,"NUMERIC(10,2)",1,None,0


----------------------------------------------------------------


### 0.2 Observations
Our database constis of 11 Table

  * **Table 1 : Album**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `AlbumId`      | INTEGER | 1 | NONE | 1|
| `Title` | NVARCHAR(160)	 | 1 | NONE | 0|
| `ArtistId`| INTEGER | 1| NONE | 0 |

  * **Table 2 : Artist**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `ArtistId`      | INTEGER | 1 | NONE | 1|
| `Name` | NVARCHAR(120)	 | 0 | NONE | 0|

  * **Table 3 : Customer**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `CustomerId`      | INTEGER | 1 | NONE | 1|
| `FirstName` | NVARCHAR(40)	 | 1 | NONE | 0|
| `LastName`      | NVARCHAR(20)	 | 1 | NONE | 0|
| `Company	` | NVARCHAR(80)		 | 0 | NONE | 0|
| `Address`      | 	NVARCHAR(70) | 0 | NONE | 0|
| `City` | NVARCHAR(40)		 | 0 | NONE | 0|
| `State`      | NVARCHAR(40)	| 0 | NONE | 0|
| `Country` | NVARCHAR(40)	 | 0 | NONE | 0|
| `PostalCode`      | NVARCHAR(10) | 0 | NONE | 0|
| `Phone` | NVARCHAR(24)		 | 0 | NONE | 0|
| `Fax`      | NVARCHAR(24)	 | 0 | NONE | 0|
| `Email` | NVARCHAR(60)	 | 1 | NONE | 0|
| `SupportRepId` | INTEGER	 | 0 | NONE | 0|

* **Table 4 : Employee**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `EmployeeId`      | INTEGER | 1 | NONE | 1|
| `FirstName` | NVARCHAR(20)| 1 | NONE | 0|
| `LastName`      | NVARCHAR(20)	 | 1 | NONE | 0|
| `Title	` | NVARCHAR(30)			 | 0 | NONE | 0|
| `ReportsTo	`      | 	INTEGER | 0 | NONE | 0|
| `BirthDate` | DATETIME		 | 0 | NONE | 0|
| `HireDate`      | DATETIME	| 0 | NONE | 0|
| `Address` | NVARCHAR(70)	 | 0 | NONE | 0|
| `City`      | NVARCHAR(40)	 | 0 | NONE | 0|
| `State` | NVARCHAR(40)			 | 0 | NONE | 0|
| `Country`      | NVARCHAR(40)		 | 0 | NONE | 0|
| `PostalCode` | NVARCHAR(10)	 | 0 | NONE | 0|
| `Phone` | NVARCHAR(24)		 | 0 | NONE | 0|
| `Fax` | NVARCHAR(24)		 | 0 | NONE | 0|
| `Email` | NVARCHAR(60)	 | 0 | NONE | 0|

* **Table 5 : Genre**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `GenreId`      | INTEGER | 1 | NONE | 1|
| `Name` | NVARCHAR(120)| 0 | NONE | 0|

* **Table 6 : Invoice**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `InvoiceId`      | INTEGER | 1 | NONE | 1|
| `CustomerId` | INTEGER| 1 | NONE | 0|
| `InvoiceDate`      | DATETIME	 | 1 | NONE | 0|
| `BillingAddress		` | NVARCHAR(70)				 | 0 | NONE | 0|
| `BillingCity	`      | 	NVARCHAR(40)	 | 0 | NONE | 0|
| `BillingState` | NVARCHAR(40)		 | 0 | NONE | 0|
| `BillingCountry`      | NVARCHAR(40)	| 0 | NONE | 0|
| `BillingPostalCode` | NVARCHAR(10)	 | 0 | NONE | 0|
| `Total` | NUMERIC(10,2)			 | 1 | NONE | 0|

* **Table 7 : InvoiceLine**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `InvoiceLineId`      | INTEGER | 1 | NONE | 1|
| `InvoiceId` | INTEGER| 1 | NONE | 0|
| `TrackId`      | INTEGER	 | 1 | NONE | 0|
| `UnitPrice` | NUMERIC(10,2)	| 1 | NONE | 0|
| `Quantity	`      | 	INTEGER	 | 1 | NONE | 0|

* **Table 8 : MediaType**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `MediaTypeId`      | INTEGER | 1 | NONE | 1|
| `Name` | NVARCHAR(120)	| 0 | NONE | 0|

* **Table 9 : Playlist**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `PlaylistId`      | INTEGER | 1 | NONE | 1|
| `Name` | NVARCHAR(120)	| 0 | NONE | 0|

* **Table 10 : PlaylistTrack**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `PlaylistId`      | INTEGER | 1 | NONE | 1|
| `TrackId` | INTEGER	| 1 | NONE | 2|

* **Table 11 : Track**  

| **Col_Name**       | **Type**                                | **NotNULL** | **DefaultValue** |Primary_Key|
|-------------------|---------------------------------------------|-------------------------------------------------|-----------------------------|-------------------|
| `TrackId`      | INTEGER | 1 | NONE | 1|
| `Name` | NVARCHAR(200)	| 1 | NONE | 0|
| `AlbumId` | INTEGER	| 0 | NONE | 0|
| `MediaTypeId` | INTEGER	| 1 | NONE | 0|
| `GenreId` | INTEGER	| 0 | NONE | 0|
| `Composer` | NVARCHAR(220)	| 0 | NONE | 0|
| `Milliseconds` | INTEGER	| 1 | NONE | 0|
| `Bytes` | INTEGER	| 0 | NONE | 0|
| `UnitPrice` | NUMERIC(10,2)		| 1 | NONE | 0|




## **0.3 Dataset Overview**

This dataset consists of multiple tables representing different aspects of a global music store's operations. Below is a brief description of each table, along with its key columns and purpose:

1.  **🧾 Invoice**
* **Key Columns** : `InvoiceId`, `CustomerId`, `InvoiceDate`, `BillingAddress`, `BillingCity`, `BillingState`, `BillingCountry`, `BillingPostalCode`, `Total`
* **Purpose** : Tracks individual sales transactions, including billing details, customer ID, and total purchase amount.

2.  📦 InvoiceLine
* **Key Columns**: `InvoiceLineId`, `InvoiceId`, `TrackId`, `Quantity`, `UnitPrice`
* **Purpose**: Provides line-item details for each invoice, including the tracks purchased, quantities, and price per unit.

3.  👤 Customer
* **Key Columns**: `CustomerId`, `FirstName`, `LastName`, `Company`, `Address`, `City`, `State`, `Country`, `PostalCode`, `Phone`, `Fax`, `Email`, `SupportRepId`
* **Purpose**: Stores customer details, including their contact information and location.

4.  🎵 Track
* **Key Columns**: `TrackId`, `Name`, `AlbumId`, `MediaTypeId`, `GenreId`, `Composer`, `Milliseconds`, `Bytes`, `UnitPrice`
* **Purpose**:  Contains details about individual tracks, such as their name, associated album, genre, and pricing.

5.  🎤 Artist
* **Key Columns** : `ArtistId`,`Name`
* **Purpose**: Lists music artists and their corresponding IDs.

6.  💿 Album
* **Key Columns**: `AlbumId`, `Title`, `ArtistId`
* **Purpose**: Stores album information, including album titles and associated artists.

7.  🎼 Genre
* **Key Columns**: `GenreId`, `Name`
* **Purpose**: Categorizes tracks into specific music genres.

8.  🏢 Employee
* **Key Columns**: `EmployeeId`, `LastName`, `FirstName`, `Title`, `ReportsTo`, `BirthDate`, `HireDate`, `Address`, `City`, `State`, `Country`, `PostalCode`, `Phone`, `Fax`, `Email`
* **Purpose**: Maintains information about employees, including their personal details, titles, and reporting hierarchy.

9. 💬 MediaType
* **Key Columns**: `MediaTypeId`, `Name`
* **Purpose**: Defines the format or type of media associated with each track (e.g., audio file types).

10. 📑 Playlist
* **Key Columns**: `PlaylistId`, `Name`
* **Purpose** : Represents collections of tracks grouped into playlists.

11. 🔗 PlaylistTrack
* **Key Columns**: `PlaylistId`, `TrackId`
* **Purpose**: Acts as a bridge table linking tracks to playlists, enabling many-to-many relationships.
---

**Database Relationship overview**
1.  🎤 Artist ↔ 💿 Album
  * Relation: One Artist can have many Albums.
  * Key Link: `ArtistId`

2.  💿 Album ↔ 🎵 Track
  * Relation: One Album contains many Tracks.
  * Key Link: `AlbumId`

3.  🎵 Track ↔ 🎼 Genre
  * **Relation**: Each Track belongs to one Genre.
  * **Key Link**: `GenreId`

4.  🎵 Track ↔ 💬 MediaType
  * **Relation**: Each Track has one Media Type (e.g., MP3, WAV).
  * **Key Link**: `MediaTypeId`

5.  🎵 Track ↔ 📦 InvoiceLine
  * **Relation**  : Each Track can appear in many Invoice Lines (if purchased multiple times).
  * **Key Link**: `TrackId`

6.  🧾 Invoice ↔ 📦 InvoiceLine
  * **Relation**: Each Invoice can have multiple Invoice Lines.
  * **Key Link**: `InvoiceId`

7.  🧾 Invoice ↔ 👤 Customer
  * **Relation**: Each Invoice belongs to one Customer.
  * **Key Link**: `CustomerId`

8.  👤 Customer ↔ 🏢 Employee
  * **Relation**: Each Customer is supported by one Employee (SupportRep).
  * **Key Link**: `SupportRepId`
  
9.  📑 Playlist ↔ 🎵 Track
  * **Relation**: A Playlist can have many Tracks (via PlaylistTrack table).
  * **Key Link**: PlaylistId ↔ TrackId (through 📑 PlaylistTrack)

10. 🎵 Track ↔ 📑 PlaylistTrack
  * **Relation**: Each Track can appear in many Playlists.
  * **Key Link**: `TrackId`

# 📌 **1. Requirement 1**
***Task Name :*** Complex Joins and CTEs

***Description :***
  * Use the database (chinook-database)
  * Use INNER JOIN and LEFT JOIN to combine the Customer, Invoice, and  InvoiceLine tables.
  * Use a CTE (Common Table Expression) to calculate the total amount spent by each customer.
  * Calculate the top 10 customers by total spending.

## **1.1 Use INNER JOIN and LEFT JOIN to combine the Customer, Invoice, and InvoiceLine tables.**

In [44]:
# Make a dictionary to Track the query execution time
Track_Query_Time = {}

In [45]:
query = '''
SELECT
    Customer.CustomerId,
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    Invoice.InvoiceId,
    Invoice.Total
FROM Customer
INNER JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
LEFT JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
LIMIT 10;
'''

# Execute Query and Display Results
try:
    start_time = time.time()  # Record the start time
    df_joins = pd.read_sql_query(query, connection)
    end_time = time.time()  # Record the end time
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    # Add the execution time to the tracker
    Track_Query_Time['INNER JOIN and LEFT JOIN'] = execution_time
    print(f"✅ INNER and LEFT JOIN query  successfully in {execution_time:.2f} milliseconds")
    display(df_joins)
except Exception as e:
    print(f"❌ Error executing join query: {e}")

✅ INNER and LEFT JOIN query  successfully in 3.85 milliseconds


,CustomerId,FullName,InvoiceId,Total
0,2,Leonie Köhler,1,1.98
1,2,Leonie Köhler,1,1.98
2,4,Bjørn Hansen,2,3.96
3,4,Bjørn Hansen,2,3.96
4,4,Bjørn Hansen,2,3.96
5,4,Bjørn Hansen,2,3.96
6,8,Daan Peeters,3,5.94
7,8,Daan Peeters,3,5.94
8,8,Daan Peeters,3,5.94
9,8,Daan Peeters,3,5.94


## **1.2 Use a CTE (Common Table Expression) to calculate the total amount spent by each customer.**

In [46]:
query = '''
SELECT
  Customer.CustomerId,
  Customer.FirstName || ' ' || Customer.LastName AS FullName,
  SUM(Invoice.Total) AS TotalSpent
FROM Customer
INNER JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY TotalSpent DESC
'''
# Execute Query and Display Results
try:
    start_time = time.time()
    df_customer_spending = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)
    Track_Query_Time['CTE to calculate totalspend'] = execution_time
    print(f"✅ CTE query executed successfully in {execution_time:.2f} milliseconds")
    display(df_customer_spending)
except Exception as e:
    print(f"❌ Error executing CTE query: {e}")

✅ CTE query executed successfully in 3.47 milliseconds


,CustomerId,FullName,TotalSpent
0,6,Helena Holý,49.62
1,26,Richard Cunningham,47.62
2,57,Luis Rojas,46.62
3,45,Ladislav Kovács,45.62
4,46,Hugh O'Reilly,45.62
5,28,Julia Barnett,43.62
6,24,Frank Ralston,43.62
7,37,Fynn Zimmermann,43.62
8,7,Astrid Gruber,42.62
9,25,Victor Stevens,42.62


## **1.3 Calculate the top 10 customers by total spending.**

In [47]:
query = '''
SELECT
  Customer.CustomerId,
  Customer.FirstName || ' ' || Customer.LastName AS FullName,
  SUM(Invoice.Total) AS TotalSpent
FROM Customer
INNER JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY TotalSpent DESC
'''



# Execute Query and Display Results
try:
    start_time = time.time()
    df_customer_spending = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['CTE to calculate TOP 10 customers by total spending'] = execution_time
    print(f"✅ CTE query executed successfully in {execution_time:.2f} milliseconds")
    display(df_customer_spending[:10])
except Exception as e:
    print(f"❌ Error executing CTE query: {e}")

✅ CTE query executed successfully in 5.23 milliseconds


,CustomerId,FullName,TotalSpent
0,6,Helena Holý,49.62
1,26,Richard Cunningham,47.62
2,57,Luis Rojas,46.62
3,45,Ladislav Kovács,45.62
4,46,Hugh O'Reilly,45.62
5,28,Julia Barnett,43.62
6,24,Frank Ralston,43.62
7,37,Fynn Zimmermann,43.62
8,7,Astrid Gruber,42.62
9,25,Victor Stevens,42.62


In [48]:
## ADD SOME VISUALIZATIONS
fig = px.bar(df_customer_spending,
             x='FullName',
             y='TotalSpent',
             title='Top Customers by Total Spending')
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'method': 'update',
                    'label': 'Top 5 Genres',
                    'args': [{'x': [df_customer_spending['FullName'][:5]], 'y': [df_customer_spending['TotalSpent'][:5]]}]
                },
                {
                    'method': 'update',
                    'label': 'Top 10 Genres',
                    'args': [{'x': [df_customer_spending['FullName'][:10]], 'y': [df_customer_spending['TotalSpent'][:10]]}]
                },
                {
                    'method': 'update',
                    'label': 'All Genres',
                    'args': [{'x': [df_customer_spending['FullName']], 'y': [df_customer_spending['TotalSpent']]}]
                },

            ],
            'direction': 'down',
            'showactive': True
        }
    ]
)

# Save it to html file
fig.write_html("01_03_top_10_customers.html")
# Make the title at the midle with blue color
fig.update_layout(title_x=0.5, title_font=dict(color='darkred'))
# Show the bar plot
fig.show()


# 📌 **2. Requirement 2**
***Task Name :*** Window Functions for Ranking

***Description :***
  * Use window functions to rank products by total sales (using the InvoiceLine and Track tables).
  * Calculate the rank of each product by total sales amount.
  * Identify the top-selling products in the database. Use RANK() or ROW_NUMBER() to rank the top-selling products based on the total sales (quantity sold).

## 2.1 Calculate the rank of each product by total sales amount.

In [49]:
query = '''
SELECT
    Track.Name AS ProductName,
    SUM(InvoiceLine.Quantity) AS TotalQuantitySold,
    RANK() OVER (ORDER BY SUM(InvoiceLine.Quantity) DESC) AS SalesRank
FROM InvoiceLine
INNER JOIN Track ON InvoiceLine.TrackId = Track.TrackId
GROUP BY Track.TrackId
ORDER BY TotalQuantitySold DESC
LIMIT 10;

'''

# Execute the Query and Display Result
try:
    start_time = time.time()
    df_product_sales_rank = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Window Function'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_product_sales_rank)
except Exception as e:
    print(f"❌ Error executing query: {e}")

✅ Query executed successfully in 6.64 milliseconds


,ProductName,TotalQuantitySold,SalesRank
0,Balls to the Wall,2,1
1,Inject The Venom,2,1
2,Snowballed,2,1
3,Overdose,2,1
4,Deuces Are Wild,2,1
5,Not The Doctor,2,1
6,Por Causa De Você,2,1
7,Welcome Home (Sanitarium),2,1
8,Snowblind,2,1
9,Cornucopia,2,1


In [50]:
# Make Some Visualizations
fig = px.bar(
    df_product_sales_rank,
    x='ProductName',
    y='TotalQuantitySold',
    color='SalesRank',  # Color based on SalesRank
    title='Top Products by Total Sales',
    color_continuous_scale= 'electric'  # Optional: Choose a color scale
)

# Center the Title and Adjust Font Color
fig.update_layout(
    title_x=0.5,
    title_font=dict(color='darkred'),
    coloraxis_colorbar=dict(title='Sales Rank')  # Add color legend
)
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'method': 'update',
                    'label': 'Top 5 Genres',
                    'args': [{'x': [df_customer_spending['FullName'][:5]], 'y': [df_customer_spending['TotalSpent'][:5]]}]
                },
                {
                    'method': 'update',
                    'label': 'Top 10 Genres',
                    'args': [{'x': [df_customer_spending['FullName'][:10]], 'y': [df_customer_spending['TotalSpent'][:10]]}]
                },
                {
                    'method': 'update',
                    'label': 'All Genres',
                    'args': [{'x': [df_customer_spending['FullName']], 'y': [df_customer_spending['TotalSpent']]}]
                },

            ],
            'direction': 'down',
            'showactive': True
        }
    ],
    coloraxis_colorbar=dict(title='Sales Rank')  # Add color legend
)
# Save it to an HTML file
fig.write_html("02_01_top_10_products_by_sales.html")

# Show the bar plot
fig.show()


## 2.2 Identify the top-selling products in the database. Use RANK() or ROW_NUMBER() to rank the top-selling products based on the total sales (quantity sold).

In [51]:
# SQL Query
query = '''
SELECT
    Track.Name AS ProductName,
    SUM(InvoiceLine.Quantity) AS TotalQuantitySold,
    SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales,
    RANK() OVER (ORDER BY SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) DESC) AS SalesRank
FROM InvoiceLine
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
GROUP BY Track.TrackId
ORDER BY TotalSales DESC
LIMIT 15;
'''

# Execute Query and Display Results
try:
    print("Top 15 Products by Total Sales (Ranked):")
    start_time = time.time()
    df_top_15_products = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate execution time
    Track_Query_Time['Product Sales Ranking'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_top_15_products)
except Exception as e:
    print(f"❌ Error executing query: {e}")


Top 15 Products by Total Sales (Ranked):
✅ Query executed successfully in 4.76 milliseconds


,ProductName,TotalQuantitySold,TotalSales,SalesRank
0,The Woman King,2,3.98,1
1,The Fix,2,3.98,1
2,Walkabout,2,3.98,1
3,Hot Girl,2,3.98,1
4,Gay Witch Hunt,2,3.98,1
5,Phyllis's Wedding,2,3.98,1
6,How to Stop an Exploding Man,2,3.98,1
7,Pilot,2,3.98,1
8,Occupation / Precipice,1,1.99,9
9,"Exodus, Pt. 1",1,1.99,9


In [52]:
import plotly.express as px

# Visualization: Top 15 Products by Total Sales with Ranking
fig = px.bar(
    df_top_15_products,
    x='ProductName',
    y='TotalSales',
    color='SalesRank',
    text='SalesRank',
    title='Top 15 Products by Total Sales (Ranked)',
    labels={'ProductName': 'Product', 'TotalSales': 'Total Sales ($)', 'SalesRank': 'Rank'}
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Product', yaxis_title='Total Sales ($)')
fig.update_layout(title_x=0.5, title_font=dict(color='darkred') )

# Save and Show
fig.write_html('02_02_Product_Sales_Ranking.html')
fig.show()


# 📌 **3. Requirement 3**
***Task Name :*** Indexing and Performance Optimization

***Description :***
  *  Identify the most commonly queried columns CustomerId in the Invoice table.
  * Create indexes on these columns and compare query performance with and without indexing.

  * Write a query that lists total sales for each customer, and optimize it using indexing.

## 3.1 Create indexes on these column and compare query performance with and without indexing

In [53]:
query = '''
CREATE INDEX IF NOT EXISTS idx_CustomerId ON Invoice (CustomerId);
'''
# Execute the query and display results
try:
    start_time = time.time()
    cursor.execute(query)
    connection.commit()
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    print(f"✅ Index Query executed successfully in {execution_time:.2f} milliseconds")
except Exception as e:
    print(f"❌ Error creating index: {e}")

✅ Index Query executed successfully in 0.12 milliseconds


## 3.2 Write a query that lists total sales for each customer, and optimize it using indexing.

In [54]:
query = '''
SELECT
    Customer.CustomerId,
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    SUM(Invoice.Total) AS TotalSpent
FROM Customer
INNER JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY TotalSpent DESC
LIMIT 10;
'''

# Execute Query and Display Results
try:
    start_time = time.time()
    df_optimized_query = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Optimized Query to get total sales'] = execution_time
    print(f"✅ Optimized query executed in {execution_time:.2f} milliseconds")
    display(df_optimized_query)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

✅ Optimized query executed in 2.60 milliseconds


,CustomerId,FullName,TotalSpent
0,6,Helena Holý,49.62
1,26,Richard Cunningham,47.62
2,57,Luis Rojas,46.62
3,45,Ladislav Kovács,45.62
4,46,Hugh O'Reilly,45.62
5,28,Julia Barnett,43.62
6,24,Frank Ralston,43.62
7,37,Fynn Zimmermann,43.62
8,7,Astrid Gruber,42.62
9,25,Victor Stevens,42.62


## **3.3 Write a query that makes window after indexing**

In [55]:
query = '''
 SELECT Track.Name AS ProductName, SUM(InvoiceLine.Quantity) AS TotalQuantitySold,
           RANK() OVER (ORDER BY SUM(InvoiceLine.Quantity) DESC) AS Rank
    FROM InvoiceLine
    JOIN Track ON InvoiceLine.TrackId = Track.TrackId
    GROUP BY Track.TrackId
    ORDER BY TotalQuantitySold DESC
    LIMIT 10;
'''
# Execute Query and Display Results
try:
    start_time = time.time()
    df_optimized_query_window = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Optimized Query with Window'] = execution_time
    print(f"✅ Optimized query with window executed in {execution_time:.2f} milliseconds")
    display(df_optimized_query_window)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

✅ Optimized query with window executed in 5.29 milliseconds


,ProductName,TotalQuantitySold,Rank
0,Balls to the Wall,2,1
1,Inject The Venom,2,1
2,Snowballed,2,1
3,Overdose,2,1
4,Deuces Are Wild,2,1
5,Not The Doctor,2,1
6,Por Causa De Você,2,1
7,Welcome Home (Sanitarium),2,1
8,Snowblind,2,1
9,Cornucopia,2,1


## **3.4 Summary Table**

In [56]:
# Print our summary table using Track_Query_Time
summary_table = pd.DataFrame(list(Track_Query_Time.items()), columns=['Query', 'Execution Time (milliseconds)'])
summary_table

,Query,Execution Time (milliseconds)
0,INNER JOIN and LEFT JOIN,3.85
1,CTE to calculate totalspend,3.47
2,CTE to calculate TOP 10 customers by total spe...,5.23
3,Window Function,6.64
4,Product Sales Ranking,4.76
5,Optimized Query to get total sales,2.60
6,Optimized Query with Window,5.29


In [57]:
# Compare the percentage of the time before and after
time_taken_before = summary_table.loc[summary_table['Query'] == 'INNER JOIN and LEFT JOIN', 'Execution Time (milliseconds)'].values[0]
time_taken_after = summary_table.loc[summary_table['Query'] == 'Optimized Query to get total sales', 'Execution Time (milliseconds)'].values[0]
percentage_improvement = ((time_taken_before - time_taken_after) / time_taken_before) * 100
print(f'''
✅ Observations after Indexing:
- Query execution time decreased significantly.
- Indexing allowed faster data retrieval for CustomerId in Invoice.
- The database used the index to avoid scanning all rows in the table.
- The Percentage improvment {percentage_improvement} %
''')



✅ Observations after Indexing:
- Query execution time decreased significantly.
- Indexing allowed faster data retrieval for CustomerId in Invoice.
- The database used the index to avoid scanning all rows in the table.
- The Percentage improvment 32.467532467532465 %



## 3.5 Verify Index Performance

In [58]:
# Compare before and after indexing in Top 10 Customers
time_taken_before = summary_table.loc[summary_table['Query'] == 'CTE to calculate TOP 10 customers by total spending', 'Execution Time (milliseconds)'].values[0]
time_taken_after = summary_table.loc[summary_table['Query'] == 'Optimized Query with Window', 'Execution Time (milliseconds)'].values[0]
if time_taken_before > time_taken_after:
    print("\n✅ Indexing improved the performance of the Top 10 Customer function query.")
else:
    print("\n⚠️ Indexing did not significantly improve the performance of the window function query.")

# Compare before and after indexing in window
time_taken_before = summary_table.loc[summary_table['Query'] == 'Window Function', 'Execution Time (milliseconds)'].values[0]
time_taken_after = summary_table.loc[summary_table['Query'] == 'Optimized Query with Window', 'Execution Time (milliseconds)'].values[0]
if time_taken_before > time_taken_after:
    print("\n✅ Indexing improved the performance of the window function query.")
else:
    print("\n⚠️ Indexing did not significantly improve the performance of the window function query.")



⚠️ Indexing did not significantly improve the performance of the window function query.

✅ Indexing improved the performance of the window function query.


## **3.6 🛠️ Why Indexing Sometimes Increases Query Time?**

1.  Index Overhead:
  * When creating or updating an index, the database engine must maintain and update the index structure
  * If the dataset is `small`, the overhead of maintaining an index might outweigh the benefits of using it.
2.  Query Optimizer Decision:
  * The database optimizer decides whether to use an index based on query patterns and table statistics.
  * In some cases, the optimizer might decide not to use the index if a full table scan is deemed faster.
3.  Index Creation Time:
  * The time taken to create the index is included in the timing summary. This is a one-time cost, but it can make indexing seem slower initially.
4.  Query Specificity:
  * Indexes are most beneficial when filtering or searching large datasets with selective queries.
  * If the query fetches most of the rows (e.g., no significant filtering), indexing offers little benefit.

# 🚀 **4. Requirement 4 : New Requirements and Ideas**
1. 📊 Customer Purchase Trends Over Time
2. 🔄 Product Affinity Analysis
3. 🎯 Customer Segmentation (RFM Analysis)
4. 🌍 Geographical Sales Insights
5. Sales by Genre or Artist
6. ⚙️ Query Optimization Deep Dive
7. 🛠️ Dynamic Query Parameterization
8. 📈 Interactive Dashboard Integration

## 4.1 📊 Customer Purchase Trends Over Time

* **Title**: Analyzing Customer Purchase Trends Across Time
* **Objective**: Understand monthly or yearly sales trends and identify peak purchasing periods.

---

**Query Explaination**

1. **Table Involved**
  * `Customer` : Contain Customer details
  * `Invoice` : Contains Invoice details linked to this customer
2. **Columns Selected**
  * `CustomerID` : Unique identifier for each customer
  * `FullName` : Concatenates `FirstName` and `LastName` AS `FullName`
  * `Month` : Extracted from `InvoiceDate` using `strftime('%Y-%m')
  * `TotalSpent` : Total Spend per each customer per month using `SUM(Invoice.Total).
3. **JOIN**
  * `JOIN` between `Customer` and `Invoice` on `CustomerID`
4. **GROUPING**
  * Grouped by `CustomerID` and `Month` to get monthly totals per customer
5. **ORDERING**
  * Ordered by `MONTH` ascending and `TotalSpent` descending
6. **LIMIT 15**
  * Display only the top 15 rows
---
**Purpose**
* Analyze customer spending patterns on a monthly basis.
* Identify customers who spent the most in specific months.
---
**Query Code**
```
query = '''
SELECT
    Customer.CustomerId,
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    strftime('%Y-%m', Invoice.InvoiceDate) AS Month,
    SUM(Invoice.Total) AS TotalSpent
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId, Month
ORDER BY Month ASC, TotalSpent DESC
LIMIT 15;
'''
```





In [59]:
query = '''
SELECT
    Customer.CustomerId,
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    strftime('%Y-%m', Invoice.InvoiceDate) AS Month,
    SUM(Invoice.Total) AS TotalSpent
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId, Month
ORDER BY Month ASC, TotalSpent DESC
LIMIT 15;
'''

# Execute Query and Display Results
try:
    print("Top 15 Monthly Customer Purchase Trends:")
    start_time = time.time()
    df_top_15_Monthly_Customer = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Monthly Customer Purchase Trends'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_top_15_Monthly_Customer)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

Top 15 Monthly Customer Purchase Trends:
✅ Query executed successfully in 3.13 milliseconds


,CustomerId,FullName,Month,TotalSpent
0,23,John Gordon,2009-01,13.86
1,14,Mark Philips,2009-01,8.91
2,8,Daan Peeters,2009-01,5.94
3,4,Bjørn Hansen,2009-01,3.96
4,2,Leonie Köhler,2009-01,1.98
5,37,Fynn Zimmermann,2009-01,0.99
6,2,Leonie Köhler,2009-02,13.86
7,52,Emma Jones,2009-02,8.91
8,46,Hugh O'Reilly,2009-02,5.94
9,42,Wyatt Girard,2009-02,3.96


In [60]:
# Execute Query and Display Results
try:
    print("Top 15 Monthly Customer Purchase Trends:")
    start_time = time.time()
    df_top_15_Monthly_Customer = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Monthly Customer Purchase Trends'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_top_15_Monthly_Customer)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

Top 15 Monthly Customer Purchase Trends:
✅ Query executed successfully in 3.84 milliseconds


,CustomerId,FullName,Month,TotalSpent
0,23,John Gordon,2009-01,13.86
1,14,Mark Philips,2009-01,8.91
2,8,Daan Peeters,2009-01,5.94
3,4,Bjørn Hansen,2009-01,3.96
4,2,Leonie Köhler,2009-01,1.98
5,37,Fynn Zimmermann,2009-01,0.99
6,2,Leonie Köhler,2009-02,13.86
7,52,Emma Jones,2009-02,8.91
8,46,Hugh O'Reilly,2009-02,5.94
9,42,Wyatt Girard,2009-02,3.96


**Make some Visualization**

* X-axis: Months (YYYY-MM).
* Y-axis: Total spending ($).
* Lines: Each line represents a customer’s spending trend across months.
* Markers: Highlight monthly spending points.

In [61]:
# Plot Monthly Spending Trends with Plotly Express
fig = px.line(
    df_top_15_Monthly_Customer,
    x='Month',
    y='TotalSpent',
    color='FullName',
    markers=True,
    title='Monthly Customer Purchase Trends',
    labels={
        'Month': 'Month (YYYY-MM)',
        'TotalSpent': 'Total Spending ($)',
        'FullName': 'Customer'
    }
)

fig.update_layout(
    xaxis_title='Month (YYYY-MM)',
    yaxis_title='Total Spending ($)',
    legend_title='Customers',
    xaxis=dict(tickangle=-45)
)
# Make the title in the middle
fig.update_layout(title_x=0.5)
# Save to html file
fig.write_html("04_01_Monthly_Customer_Purchase_Trends.html")


# Show the figure
fig.show()

## 4.2 🔄 Product Affinity Analysis

* **Title**: Product Synergy: Identifying Frequently Co-Purchased Products
* **Objective**:  Discover product pairs commonly bought together in a single transaction..
---
**Query Explaination**
1.  **Tables Involved**
  * `InvoiceLine` : Contains sales data (quantity and price)
  * `Track` : Links between data to specific track
  * `Genre` : Represent product category
2.  **Columns Selected**
  * `Category` : Extracted from `Genre.Name`
  * `TotalQuantitySold` : Total Unit sold per category extracted from `SUM(InvoiceLine.Quantity)`
  * `TotalRevenue`: Total sales revenue per category extracted from `SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity)`
3.  **Joins**
  * `JOIN` between `InvoiceLine` and `Track` using `TrackID`
  * `JOIN` between `Track` and `Genere` using `GenreID`
4.  **Grouping**
  * Grouped by `Genre.Name` to calculate total sales per category
5.  **Ordering**
  * Ordered by `TotalRevenue` descending order
6.  **LIMIT 15**
  * Display only top 15 value
---
**Purpose**
* Understand which product categories drive the most sales and revenue.
* Identify top-performing categories.
---
**Query Code**


```
query = '''
SELECT
  Genre.Name AS Category,
  SUM(InvoiceLine.Quantity) AS TotalQuantitySold,
  SUM(InvoiceLine.Quantity * InvoiceLine.UnitPrice) AS TotalRevenue
FROM InvoiceLine
JOIN Track on InvoiceLine.TrackId = Track.TrackId
JOIN Genre on Track.GenreId = Genre.GenreID
GROUP BY Category
ORDER BY TotalRevenue DESC
LIMIT 15
'''
```




In [62]:
query = '''
SELECT
  Genre.Name AS Category,
  SUM(InvoiceLine.Quantity) AS TotalQuantitySold,
  SUM(InvoiceLine.Quantity * InvoiceLine.UnitPrice) AS TotalRevenue
FROM InvoiceLine
JOIN Track on InvoiceLine.TrackId = Track.TrackId
JOIN Genre on Track.GenreId = Genre.GenreID
GROUP BY Category
ORDER BY TotalRevenue DESC
LIMIT 15
'''


# Execute Query and Display Results
try:
    print("Top 15 Product Categories by Sales and Revenue:")
    start_time = time.time()
    df_product_category_sales = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Product Category Sales Distribution'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_product_category_sales)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

Top 15 Product Categories by Sales and Revenue:
✅ Query executed successfully in 4.32 milliseconds


,Category,TotalQuantitySold,TotalRevenue
0,Rock,835,826.65
1,Latin,386,382.14
2,Metal,264,261.36
3,Alternative & Punk,244,241.56
4,TV Shows,47,93.53
5,Jazz,80,79.20
6,Blues,61,60.39
7,Drama,29,57.71
8,R&B/Soul,41,40.59
9,Classical,41,40.59


Make some Visualizations

* X-axis: Represents product categories.
* Y-axis: Shows total revenue generated by each category.
* Color Gradient: Represents total units sold (TotalQuantitySold).
* Bar Size: Indicates revenue contribution by category.
* Interactive Tooltips: Provide category name, revenue, and quantity sold on hover.

In [63]:
# Plot Product Category Sales Distribution with Plotly Express
fig = px.bar(
    df_product_category_sales,
    x='Category',
    y='TotalRevenue',
    color='TotalQuantitySold',
    title='Top 15 Product Categories by Sales and Revenue',
    labels={
        'Category': 'Product Category',
        'TotalRevenue': 'Total Revenue ($)',
        'TotalQuantitySold': 'Total Quantity Sold'
    }
)

fig.update_layout(
    xaxis=dict(tickangle=-45),
    title_x=0.5

)
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'method': 'update',
                    'label': 'Top 5 Genres',
                    'args': [{'x': [df_product_category_sales['Category'][:5]], 'y': [df_product_category_sales['TotalRevenue'][:5]]}]
                },
                {
                    'method': 'update',
                    'label': 'Top 10 Genres',
                    'args': [{'x': [df_product_category_sales['Category'][:10]], 'y': [df_product_category_sales['TotalRevenue'][:10]]}]
                },
                {
                    'method': 'update',
                    'label': 'All Genres',
                    'args': [{'x': [df_product_category_sales['Category']], 'y': [df_product_category_sales['TotalRevenue']]}]
                },

            ],
            'direction': 'down',
            'showactive': True
        }
    ],
    coloraxis_colorbar=dict(title='TotalQuantitySold')  # Add color legend
)
# Save to html file
fig.write_html("04_02_Product_Category_Sales_Distribution.html")
# show the figure
fig.show()


## 4.3 🎯 Customer Segmentation (RFM Analysis)

* **Title**: Customer Blueprint: Strategic Segmentation with RFM Analysis
* **Objective**:  Classify customers based on Recency, Frequency, and Monetary value to optimize marketing efforts.


---


**Query Explaination**
1.  **Table Involved**
  * `Customer` : Contain customer details
  * `Invoice` :  Tracks customer purchase and total spending
2.  **Columns Selected**
  * `CustomerId` : Unique identifier for each customer
  * `CustomerName` : Combine between `FirstName` and `LastName` AS `FullName`
  * `TotalPurchase` : Count of invoices per customer `(COUNT(Invoice.InvoiceID))`
  * `TotalRevnue` : Total Revenue generated by each customer `(SUM(Invoice.Total))`
3.  **Joins**
  * `JOIN` between `Customer` and `Invoice` on `CustomerId`
4.  **Grouping**
  * Grouped by `CustomerId`
5.  **Ordering**
  * Order by `TotalRevnue` in descending order
6.  **LIMIT 15**
  * Display only top 15 records


---


**Purpose**

  * Identify the most valuable customers based on their total spending and number of purchases.
  * Provide insights for targeted marketing or loyalty programs.


---
**Query Code**


```
query = '''
SELECT
  Customer.CustomerId,
  Customer.FirstName || ' ' || Customer.LastName AS FullName,
  COUNT(Invoice.InvoiceID) AS TotalPurchase,
  SUM(Invoice.Total) AS TotalRevenue
FROM Customer
JOIN Invoice ON Customer.CustomerID = Invoice.InvoiceID
GROUP BY Customer.CustomerID
ORDER BY TotalRevenue DESC
LIMIT 15;
'''
```





In [64]:
query = '''
SELECT
    Customer.CustomerId,
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    COUNT(Invoice.InvoiceId) AS TotalPurchases,
    SUM(Invoice.Total) AS TotalRevenue
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY TotalRevenue DESC
LIMIT 15;
'''
# Execute Query and Display Results
try:
    print("Top 15 High-Value Customers by Revenue and Purchase Count:")
    start_time = time.time()
    df_high_value_customers = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['High-Value Customers'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_high_value_customers)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

Top 15 High-Value Customers by Revenue and Purchase Count:
✅ Query executed successfully in 2.71 milliseconds


,CustomerId,FullName,TotalPurchases,TotalRevenue
0,6,Helena Holý,7,49.62
1,26,Richard Cunningham,7,47.62
2,57,Luis Rojas,7,46.62
3,45,Ladislav Kovács,7,45.62
4,46,Hugh O'Reilly,7,45.62
5,28,Julia Barnett,7,43.62
6,24,Frank Ralston,7,43.62
7,37,Fynn Zimmermann,7,43.62
8,7,Astrid Gruber,7,42.62
9,25,Victor Stevens,7,42.62


Make Some Visualizations
* X-axis: Represents customer names.
* Y-axis: Shows total revenue generated by each customer.
* Color Gradient: Indicates the total number of purchases.
* Bar Size: Reflects revenue contribution by each customer.
* Interactive Tooltips: Provide customer name, total revenue, and   purchase count on hover.

In [65]:
# Plot High-Value Customers with Plotly Express
fig = px.bar(
    df_high_value_customers,
    x='FullName',
    y='TotalRevenue',
    color='TotalPurchases',
    title='Top 15 High-Value Customers by Revenue and Purchase Count',
    labels={
        'CustomerName': 'Customer Name',
        'TotalRevenue': 'Total Revenue ($)',
        'TotalPurchases': 'Total Purchases'
    }
)

fig.update_layout(
    title_x=0.5,
    xaxis=dict(tickangle=-45)

)
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'method': 'update',
                    'label': 'Top 5 Genres',
                    'args': [{'x': [df_high_value_customers['FullName'][:5]], 'y': [df_high_value_customers['TotalRevenue'][:5]]}]
                },
                {
                    'method': 'update',
                    'label': 'Top 10 Genres',
                    'args': [{'x': [df_high_value_customers['FullName'][:10]], 'y': [df_high_value_customers['TotalRevenue'][:10]]}]
                },
                {
                    'method': 'update',
                    'label': 'All Genres',
                    'args': [{'x': [df_high_value_customers['FullName']], 'y': [df_high_value_customers['TotalRevenue']]}]
                },

            ],
            'direction': 'down',
            'showactive': True
        }
    ],
    coloraxis_colorbar=dict(title='TotalPurchases')  # Add color legend
)
# Save to html file
fig.write_html("04_03_High_Value_Customers.html")
# Show the figure

fig.show()


## 4.4 🌍 Geographical Sales Insights

* **Title**: Sales Atlas: Mapping Geographical Distribution of Revenue
* **Objective**:  Explore sales distribution across different cities and countries.

---
**Query Explaination**
1.  **Table Involved**
  * `Invoice` : Cotains details about sales transactions, including billing country and total sales
2.  **Columns Selected**
  * `BillingCountry` : Country where the invoice was build
  * `TotalSales` : Total sales revenue for each country `(SUM(Total))`
  * `TotalInvoices` : Count invoices per Country `(COUNT(InvoiceId))`
3.  **Joins**
  * No `JOIN` needed because all the information are in one table
4.  **Grouping**
  * Group by `BillingCountry` to aggregate date per country
5.  **Ordering**
  * Ordered by `TotalSales` in descending order
6.  **LIMIT 15**
  * Display only the top 15 countries
---
**Purpose**
  * Identify which countries contribute the most in the revenue
  * Understand geographical sales distibution to optimize regional marketing and sales efforts
---
**Query Code**


```
query = '''
SELECT
  BillingCountry AS Country,
  SUM(Total) AS TotalSales,
  COUNT(InvoiceID) AS TotalInvoices
FROM INVOICE
GROUP BY Country
ORDER BY TotalSales
LIMIT 15;
'''
```



In [66]:
query = '''
SELECT
  BillingCountry AS Country,
  SUM(Total) AS TotalSales,
  COUNT(InvoiceID) AS TotalInvoices
FROM INVOICE
GROUP BY Country
ORDER BY TotalSales DESC
LIMIT 15;
'''

# Execute Query and Display Results
try:
    print("Top 15 Countries by Total Sales:")
    start_time = time.time()
    df_sales_by_country = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)  # Calculate the time taken
    Track_Query_Time['Geographical Sales Analysis'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_sales_by_country)
except Exception as e:
    print(f"❌ Error executing optimized query: {e}")

Top 15 Countries by Total Sales:
✅ Query executed successfully in 2.00 milliseconds


,Country,TotalSales,TotalInvoices
0,USA,523.06,91
1,Canada,303.96,56
2,France,195.10,35
3,Brazil,190.10,35
4,Germany,156.48,28
5,United Kingdom,112.86,21
6,Czech Republic,90.24,14
7,Portugal,77.24,14
8,India,75.26,13
9,Chile,46.62,7


Make Some Visualizations
* X-axis: Represents customer names.
* Y-axis: Shows total revenue generated by each customer.
* Color Gradient: Indicates the total number of purchases.
* Bar Size: Reflects revenue contribution by each customer.
* Interactive Tooltips: Provide customer name, total revenue, and purchase count on hover.

In [67]:
# Plot Geographical Sales Analysis with Plotly Express
fig = px.bar(
    df_sales_by_country,
    x='Country',
    y='TotalSales',
    color='TotalInvoices',
    title='Top 15 Countries by Total Sales',
    labels={
        'Country': 'Country',
        'TotalSales': 'Total Sales ($)',
        'TotalInvoices': 'Total Invoices'
    }
)

fig.update_layout(
    xaxis_title='Country',
    yaxis_title='Total Sales ($)',
    legend_title='Total Invoices',
    xaxis=dict(tickangle=-45),
    title_x=0.5
)
# Save to html
fig.write_html("04_04_Geographical_Sales_Analysis.html")
# Show the figure
fig.show()


In [68]:
# Map country names to ISO Alpha-3 codes
try:
  import pycountry
except:
  !pip install pycountry
  import pycountry
  print("pycountry is successfully installed")

def get_iso3_code(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        print(f"❌ Country name '{country_name}' not recognized.")
        return None

# Add ISO Alpha-3 codes to DataFrame
df_sales_by_country['ISO_Code'] = df_sales_by_country['Country'].apply(get_iso3_code)

# Remove rows with invalid ISO codes
df_sales_by_country = df_sales_by_country.dropna(subset=['ISO_Code'])


In [69]:
# Draw Scatter Plot on the World Map
fig_scatter = px.scatter_geo(
    df_sales_by_country,
    locations='ISO_Code',
    size='TotalSales',
    hover_name='Country',
    title='Global Sales Distribution (Scatter Plot)',
    color='TotalSales',
    color_continuous_scale='Viridis',
    projection='orthographic'
)

# Save to HTML file
fig_scatter.write_html("04_04_Geographical_Sales_Analysis_Scatter.html")
# Show the figure
fig_scatter.show()


In [70]:
# Draw Choropleth Heatmap on the World Map
fig = px.choropleth(
    df_sales_by_country,
    locations='ISO_Code',
    color='TotalSales',
    hover_name='Country',
    title='Global Sales Distribution by Country',
    color_continuous_scale='Viridis',
    labels={
        'TotalSales': 'Total Sales ($)'
    },
    projection='orthographic'  # Orthographic view for a spherical map
)


# Save to HTML file
fig.write_html("04_04_Geographical_Sales_Analysis_Heatmap.html")
# Show the figure
fig.show()


## 4.5 🎯 Sales by Genre or Artist

* **Title**: Melodic Metrics: Analyzing Sales by Music Genre and Artist
* **Objective**:  Identify top-performing genres and artists driving sales.
---
**Query Explaination**
1.  **Table Involved**
  * `Genre` : Represent product category `Genre.Name` AS Category
  * `Track` : Links between tracks and their category through `GenreId`
  * `InvoiceLine` : Provides sales details (`UnitPrice`, `Quantity`, `TrackId`)
2.  **Columns Selected**
  * `Genre.Name` : The category of each music
  * `TotalSales` : Total sales per category `SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity)`
3.  **Joins**
  * `Genre` -> `Track` : are joined via `GenreId` to match tracks with their category
  * `Track` -> `InvoiceLine` : are joined via `TrackId` to link between sales and tracks
4.  **Grouping**
  * Grouped by `Category` to calculate totalsales per each category
5.  **Ordering**
  * Order by `TotalSales` in descending order to rank categories by their sales
6.  **LIMIT 15**
  * Display top 15 only
---
**Purpose**
* To identify the top-performing music genres based on total sales.
* Understand which genres drive revenue and dominate the market.

**Query Code**


```
query = '''
SELECT
    Genre.Name AS Category,
    SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales
FROM Genre
JOIN Track ON Genre.GenreId = Track.GenreId
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
GROUP BY Category
ORDER BY TotalSales DESC
LIMIT 15;
'''
```



In [71]:
# SQL Query for Sales by Genre or Artist
query = '''
SELECT
    Genre.Name AS Category,
    SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales
FROM Genre
JOIN Track ON Genre.GenreId = Track.GenreId
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
GROUP BY Category
ORDER BY TotalSales DESC
LIMIT 15;
'''

# Execute Query and Display Results
try:
    print("Top 15 Genres by Total Sales:")
    start_time = time.time()
    df_sales_by_genre = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)
    Track_Query_Time['Sales by Genre or Artist'] = execution_time
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_sales_by_genre)
except Exception as e:
    print(f"❌ Error executing query: {e}")


Top 15 Genres by Total Sales:
✅ Query executed successfully in 3.60 milliseconds


,Category,TotalSales
0,Rock,826.65
1,Latin,382.14
2,Metal,261.36
3,Alternative & Punk,241.56
4,TV Shows,93.53
5,Jazz,79.20
6,Blues,60.39
7,Drama,57.71
8,R&B/Soul,40.59
9,Classical,40.59


Make Some Visualizations

In [72]:
# Visualization: Top 15 Genres by Total Sales
fig = px.bar(
    df_sales_by_genre,
    x='Category',
    y='TotalSales',
    title='Top 15 Genres by Total Sales',
    color='TotalSales',
    labels={
        'Genre': 'Music Genre',
        'TotalSales': 'Total Sales ($)'
    }
)

fig.update_layout(
    xaxis_title='Music Genre',
    yaxis_title='Total Sales ($)',
    title_x=0.5,
    xaxis=dict(tickangle=-45)
)
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'method': 'update',
                    'label': 'Top 5 Genres',
                    'args': [{'x': [df_sales_by_genre['Category'][:5]], 'y': [df_sales_by_genre['TotalSales'][:5]]}]
                },
                {
                    'method': 'update',
                    'label': 'Top 10 Genres',
                    'args': [{'x': [df_sales_by_genre['Category'][:10]], 'y': [df_sales_by_genre['TotalSales'][:10]]}]
                },
                {
                    'method': 'update',
                    'label': 'All Genres',
                    'args': [{'x': [df_sales_by_genre['Category']], 'y': [df_sales_by_genre['TotalSales']]}]
                },

            ],
            'direction': 'down',
            'showactive': True
        }
    ],
    coloraxis_colorbar=dict(title='TotalSales')  # Add color legend
)
# Save to HTML file
fig.write_html("04_05_Sales_by_Genre_Analysis.html")
# Show the figure
fig.show()


## 4.6 🛠️ Dynamic Query Parameterization

* **Title**: Adaptive Analytics: Dynamic SQL Queries with User Input
* **Objective**:  Enable flexible query execution with dynamic user-defined parameters.

**Query Explaination**
1.  **Table Involved**
2.  **Columns Selected**
3.  **Joins**
4.  **Grouping**
5.  **Ordering**
6.  **LIMIT 15**

**Purpose**

**Query Code**

In [73]:
# Dynamic Query Parameters
user_genre = 'Rock'  # Example user input
user_limit = 15

# Main Query with Dynamic Parameters
query = f'''
SELECT
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    Genre.Name AS Genre,
    SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Genre ON Track.GenreId = Genre.GenreId
WHERE Genre.Name = ?
GROUP BY Customer.CustomerId, Customer.FirstName, Customer.LastName, Genre.Name
ORDER BY TotalSales DESC
LIMIT ?;
'''

# Execute Query and Display Results
try:
    print("Dynamic Query: Customer Sales by Selected Genre")
    start_time = time.time()
    df_dynamic_query = pd.read_sql_query(query, connection, params=(user_genre, user_limit))
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_dynamic_query)
except Exception as e:
    print(f"❌ Error executing dynamic query: {e}")


Dynamic Query: Customer Sales by Selected Genre
✅ Query executed successfully in 5.32 milliseconds


,FullName,Genre,TotalSales
0,Eduardo Martins,Rock,28.71
1,Robert Brown,Rock,24.75
2,Stanisław Wójcik,Rock,21.78
3,Enrique Muñoz,Rock,21.78
4,Mark Taylor,Rock,21.78
5,Daan Peeters,Rock,20.79
6,Kara Nielsen,Rock,20.79
7,Niklas Schröder,Rock,20.79
8,Michelle Brooks,Rock,18.81
9,Edward Francis,Rock,18.81


In [74]:
# Visualization: Customer Sales by Selected Genre
fig = px.bar(
    df_dynamic_query,
    x='FullName',
    y='TotalSales',
    color='Genre',
    title=f'Top {user_limit} Customers by Sales in Genre: {user_genre}',
    labels={
        'FullName': 'Customer Name',
        'TotalSales': 'Total Sales ($)',
        'Genre': 'Music Genre'
    }
)

# Adjust layout for better readability
fig.update_layout(
    xaxis_title='Customer Name',
    yaxis_title='Total Sales ($)',
    xaxis_tickangle=-45
)
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'method': 'update',
                    'label': 'Top 5 Genres',
                    'args': [{'x': [df_dynamic_query['FullName'][:5]], 'y': [df_dynamic_query['TotalSales'][:5]]}]
                },
                {
                    'method': 'update',
                    'label': 'Top 10 Genres',
                    'args': [{'x': [df_dynamic_query['FullName'][:10]], 'y': [df_dynamic_query['TotalSales'][:10]]}]
                },
                {
                    'method': 'update',
                    'label': 'All Genres',
                    'args': [{'x': [df_dynamic_query['FullName']], 'y': [df_dynamic_query['TotalSales']]}]
                },

            ],
            'direction': 'down',
            'showactive': True
        }
    ],
    coloraxis_colorbar=dict(title='Genre')  # Add color legend
)
# Save to HTML file
fig.write_html("04_06_Dynamic_Query_Parameterized_Results.html")
# Show the figure
fig.show()


## 4.7 📈 Interactive Dashboard Integration

* **Title**: Data at Your Fingertips: Building an Interactive Sales Dashboard
* **Objective**:  Present SQL insights through a user-friendly interactive dashboard interface.
---
**Query Explaination**
1.  **Table Involved**
  * `Customer`
  * `Invoice`
  * `InvoiceLine`
  * `Track`
  * `Album`
  * `Genre`
  * `Artis`
2.  **Columns Selected**
  * `FullName` : `Customer.FirstName || ' ' || Customer.LastName`
  * `Category` : The category of the track `Genre.Name AS Category`
  * `ArtistName` : `Artist.Name AS ArtistName`
  * `TotalSalesPerArtist` : The total sales per Artist `SUM(InvoiceLine.Quantity * InvoiceLine.Quantity)`
  * `TotalNumberInvoices` : `COUNT(Invoice.InvoiceId)`
  * `AvgInvoiceTotal` : `AVG(Invoice.Total)`
3.  **Joins**
  * `Customer` → `Invoice` : Are joined together via `CustomerId`
  * `Invoice` → `InvoiceLine` : Are joined together via `InvoiceId`
  * `InvoiceLine` → `Track` : Are joined together via `TrackId`
  * `Track` → `Album` : Are joined together via `AlbumId`
  * `Album` → `Artist` : Are joined together through `ArtistId`
  * `Track` → `Genre` : Are joined together through `GenreId`
4.  **Grouping**
  * Grouped By : `CustomerId`,`Category`,`ArtistName`
5.  **Ordering**
  * Ordered by `TotalSales` DESC
6.  **LIMIT 15**
  * Top 15 customers by total sales per artist and genre.
---
**Purpose**
* Create an interactive dashboard for visualizing customer, genre, and artist sales data.
* Provide dynamic filtering options for users (e.g., by genre or artist).
* Enable quick insights through aggregated sales data
---
**Query Code**


```
query = '''
SELECT
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    Genre.Name AS Category,
    Artist.Name AS ArtistName,
    SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales,
    COUNT(Invoice.InvoiceId) AS TotalInvoices,
    AVG(Invoice.Total) AS AvgInvoiceValue
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
JOIN Artist ON Album.ArtistId = Artist.ArtistId
JOIN Genre ON Track.GenreId = Genre.GenreId
GROUP BY Customer.CustomerId, Category, ArtistName
ORDER BY TotalSales DESC
LIMIT 15;
'''
```



In [75]:
# Interactive Dashboard Query
query = '''
SELECT
    Customer.FirstName || ' ' || Customer.LastName AS FullName,
    Genre.Name AS Category,
    Artist.Name AS ArtistName,
    SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales,
    COUNT(Invoice.InvoiceId) AS TotalInvoices,
    AVG(Invoice.Total) AS AvgInvoiceValue
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
JOIN Artist ON Album.ArtistId = Artist.ArtistId
JOIN Genre ON Track.GenreId = Genre.GenreId
GROUP BY Customer.CustomerId, Category, ArtistName
ORDER BY TotalSales DESC
LIMIT 15;
'''

# Execute Query and Display Results
try:
    print("Interactive Dashboard: Top 15 Customers by Artist and Genre")
    start_time = time.time()
    df_dashboard = pd.read_sql_query(query, connection)
    end_time = time.time()
    execution_time = round((end_time - start_time) * 1000, 2)
    print(f"✅ Query executed successfully in {execution_time:.2f} milliseconds")
    display(df_dashboard)
except Exception as e:
    print(f"❌ Error executing dashboard query: {e}")


Interactive Dashboard: Top 15 Customers by Artist and Genre
✅ Query executed successfully in 16.11 milliseconds


,FullName,Category,ArtistName,TotalSales,TotalInvoices,AvgInvoiceValue
0,Hugh O'Reilly,TV Shows,Lost,13.93,7,21.86
1,Helena Holý,TV Shows,Lost,9.95,5,25.86
2,Frank Ralston,Comedy,The Office,9.95,5,9.54
3,Bjørn Hansen,Rock,Led Zeppelin,8.91,9,8.91
4,Astrid Gruber,Rock,U2,8.91,9,18.86
5,Michelle Brooks,Rock,Deep Purple,8.91,9,13.86
6,Heather Leacock,Metal,Metallica,8.91,9,8.91
7,Madalena Sampaio,Rock,U2,8.91,9,8.91
8,Niklas Schröder,Rock,Deep Purple,8.91,9,13.86
9,Wyatt Girard,Metal,Metallica,8.91,9,8.91


In [76]:
# Visualization: Interactive Sales Dashboard
fig = px.sunburst(
    df_dashboard,
    path=['Category', 'ArtistName', 'FullName'],
    values='TotalSales',
    color='TotalSales',
    title='Interactive Sales Dashboard: Genre → Artist → Customer',
    color_continuous_scale='Blues',
    labels={
        'Genre': 'Music Genre',
        'Artist': 'Artist Name',
        'FullName': 'Customer Name',
        'TotalSales': 'Total Sales ($)'
    }
)

# Adjust layout for better readability
fig.update_layout(
    margin=dict(t=50, l=25, r=25, b=25)
)


# Save to HTML file
fig.write_html("04_07_Interactive_Sales_Dashboard.html")
# Show the figure
fig.show()


#4. **Finalization and Cleanup**

In [77]:
try:
   connection.close()
   print("✅ Database connection closed successfully.")
except Exception as e:
   print(f"❌ Failed to close connection: {e}")

✅ Database connection closed successfully.
